In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.naive_bayes import BernoulliNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from scipy import stats
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn import ensemble
from sklearn import tree
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_classif
import geopy.distance
from sklearn.pipeline import Pipeline
from datetime import timedelta 

In [2]:
df = pd.read_csv('nyc bus data.csv')
df = df.drop_duplicates()
# = df.dropna()
df.head()

C:\Users\sajee\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (14,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,RecordedAtTime,DirectionRef,PublishedLineName,OriginName,OriginLat,OriginLong,DestinationName,DestinationLat,DestinationLong,VehicleRef,VehicleLocation.Latitude,VehicleLocation.Longitude,NextStopPointName,ArrivalProximityText,DistanceFromStop,ExpectedArrivalTime,ScheduledArrivalTime,Unnamed: 17
0,6/1/2017 0:03,0,B8,4 AV/95 ST,40.616104,-74.031143,BROWNSVILLE ROCKAWAY AV,40.656048,-73.907379,NYCT_430,40.635170,-73.960803,FOSTER AV/E 18 ST,approaching,76,6/1/2017 0:03,24:06:14,NaN
1,6/1/2017 0:03,1,S61,ST GEORGE FERRY/S61 & S91,40.643169,-74.073494,S I MALL YUKON AV,40.575935,-74.167686,NYCT_8263,40.590802,-74.158340,MERRYMOUNT ST/TRAVIS AV,approaching,62,6/1/2017 0:03,23:58:02,NaN
2,6/1/2017 0:03,0,Bx10,E 206 ST/BAINBRIDGE AV,40.875008,-73.880142,RIVERDALE 263 ST,40.912376,-73.902534,NYCT_4223,40.886010,-73.912647,HENRY HUDSON PKY E/W 235 ST,at stop,5,6/1/2017 0:03,24:00:53,NaN
3,6/1/2017 0:03,0,Q5,TEARDROP/LAYOVER,40.701748,-73.802399,ROSEDALE LIRR STA via MERRICK,40.666012,-73.735939,NYCT_8422,40.668002,-73.729348,HOOK CREEK BL/SUNRISE HY,< 1 stop away,267,6/1/2017 0:04,24:03:00,NaN
4,6/1/2017 0:03,1,Bx1,RIVERDALE AV/W 231 ST,40.881187,-73.909340,MOTT HAVEN 136 ST via CONCOURSE,40.809654,-73.928360,NYCT_4710,40.868134,-73.893032,GRAND CONCOURSE/E 196 ST,at stop,11,6/1/2017 0:03,23:59:38,NaN


In [3]:
df.shape

(1048575, 18)

In [4]:
#subset the dataframe to include only the first 100,000 rows, or observations.
#df = df.iloc[:700000, :]
#df.shape

In [5]:
#checking for column types and missing data
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1048575 entries, 0 to 1048574
Data columns (total 18 columns):
RecordedAtTime               1048575 non-null object
DirectionRef                 1048575 non-null int64
PublishedLineName            1048575 non-null object
OriginName                   1040112 non-null object
OriginLat                    1040112 non-null float64
OriginLong                   1040112 non-null float64
DestinationName              1048575 non-null object
DestinationLat               1047329 non-null float64
DestinationLong              1047329 non-null float64
VehicleRef                   1048575 non-null object
VehicleLocation.Latitude     1048575 non-null float64
VehicleLocation.Longitude    1048575 non-null float64
NextStopPointName            1047439 non-null object
ArrivalProximityText         1047439 non-null object
DistanceFromStop             1047439 non-null object
ExpectedArrivalTime          931342 non-null object
ScheduledArrivalTime         102290

In [6]:
#filling in missing data 
df['OriginName'].ffill(axis=0, inplace=True)
df['OriginLat'].ffill(axis=0, inplace=True)
df['OriginLong'].ffill(axis=0, inplace=True)
df['DestinationLat'].ffill(axis=0, inplace=True)
df['DestinationLong'].ffill(axis=0, inplace=True)
df['DistanceFromStop'].ffill(axis=0, inplace=True)
df['ArrivalProximityText'].ffill(axis=0, inplace=True)
df['NextStopPointName'].ffill(axis=0, inplace=True)
df['ExpectedArrivalTime'].ffill(axis=0, inplace=True)
df['ScheduledArrivalTime'].ffill(axis=0, inplace=True)


In [7]:
#splitting 'ExpectedArrivalTime' column
expected_split = df['ExpectedArrivalTime'].str.split(expand=True)
print(expected_split.head())

          0     1
0  6/1/2017  0:03
1  6/1/2017  0:03
2  6/1/2017  0:03
3  6/1/2017  0:04
4  6/1/2017  0:03


In [8]:
#creating new feature; the 'expected_date' column
df['expected_date'] = expected_split[0]

In [9]:
df.head()

,RecordedAtTime,DirectionRef,PublishedLineName,OriginName,OriginLat,OriginLong,DestinationName,DestinationLat,DestinationLong,VehicleRef,VehicleLocation.Latitude,VehicleLocation.Longitude,NextStopPointName,ArrivalProximityText,DistanceFromStop,ExpectedArrivalTime,ScheduledArrivalTime,Unnamed: 17,expected_date
0,6/1/2017 0:03,0,B8,4 AV/95 ST,40.616104,-74.031143,BROWNSVILLE ROCKAWAY AV,40.656048,-73.907379,NYCT_430,40.635170,-73.960803,FOSTER AV/E 18 ST,approaching,76,6/1/2017 0:03,24:06:14,NaN,6/1/2017
1,6/1/2017 0:03,1,S61,ST GEORGE FERRY/S61 & S91,40.643169,-74.073494,S I MALL YUKON AV,40.575935,-74.167686,NYCT_8263,40.590802,-74.158340,MERRYMOUNT ST/TRAVIS AV,approaching,62,6/1/2017 0:03,23:58:02,NaN,6/1/2017
2,6/1/2017 0:03,0,Bx10,E 206 ST/BAINBRIDGE AV,40.875008,-73.880142,RIVERDALE 263 ST,40.912376,-73.902534,NYCT_4223,40.886010,-73.912647,HENRY HUDSON PKY E/W 235 ST,at stop,5,6/1/2017 0:03,24:00:53,NaN,6/1/2017
3,6/1/2017 0:03,0,Q5,TEARDROP/LAYOVER,40.701748,-73.802399,ROSEDALE LIRR STA via MERRICK,40.666012,-73.735939,NYCT_8422,40.668002,-73.729348,HOOK CREEK BL/SUNRISE HY,< 1 stop away,267,6/1/2017 0:04,24:03:00,NaN,6/1/2017
4,6/1/2017 0:03,1,Bx1,RIVERDALE AV/W 231 ST,40.881187,-73.909340,MOTT HAVEN 136 ST via CONCOURSE,40.809654,-73.928360,NYCT_4710,40.868134,-73.893032,GRAND CONCOURSE/E 196 ST,at stop,11,6/1/2017 0:03,23:59:38,NaN,6/1/2017


In [10]:
#formatting the 'ScheduledArrivalTime'
df['ScheduledArrivalTime'] = df['ScheduledArrivalTime'].apply(lambda x: x[0:2].replace('24', '00')) + df['ScheduledArrivalTime'].apply(lambda x: x[2:])

In [11]:
#examining the 'ScheduledArrivalTime' column
df['ScheduledArrivalTime'].head()

0    00:06:14
1    23:58:02
2    00:00:53
3    00:03:00
4    23:59:38
Name: ScheduledArrivalTime, dtype: object

In [12]:
#creating new feature'Scheduled_datetime' column
df['Scheduled_datetime'] = df['expected_date'] + ' ' + df['ScheduledArrivalTime']

In [13]:
#examining 'Scheduled_datetime' column
df['Scheduled_datetime'].head()

0    6/1/2017 00:06:14
1    6/1/2017 23:58:02
2    6/1/2017 00:00:53
3    6/1/2017 00:03:00
4    6/1/2017 23:59:38
Name: Scheduled_datetime, dtype: object

In [14]:
#converting 'Scheduled_datetime' to datetime object
df['Scheduled_datetime'] = pd.to_datetime(df['Scheduled_datetime'], errors='coerce')

In [15]:
#converting 'Expected_arrival' column to datetime object
df['Expected_datetime'] = pd.to_datetime(df['ExpectedArrivalTime'], errors='coerce')

In [16]:
df.head()

,RecordedAtTime,DirectionRef,PublishedLineName,OriginName,OriginLat,OriginLong,DestinationName,DestinationLat,DestinationLong,VehicleRef,...,VehicleLocation.Longitude,NextStopPointName,ArrivalProximityText,DistanceFromStop,ExpectedArrivalTime,ScheduledArrivalTime,Unnamed: 17,expected_date,Scheduled_datetime,Expected_datetime
0,6/1/2017 0:03,0,B8,4 AV/95 ST,40.616104,-74.031143,BROWNSVILLE ROCKAWAY AV,40.656048,-73.907379,NYCT_430,...,-73.960803,FOSTER AV/E 18 ST,approaching,76,6/1/2017 0:03,00:06:14,NaN,6/1/2017,2017-06-01 00:06:14,2017-06-01 00:03:00
1,6/1/2017 0:03,1,S61,ST GEORGE FERRY/S61 & S91,40.643169,-74.073494,S I MALL YUKON AV,40.575935,-74.167686,NYCT_8263,...,-74.158340,MERRYMOUNT ST/TRAVIS AV,approaching,62,6/1/2017 0:03,23:58:02,NaN,6/1/2017,2017-06-01 23:58:02,2017-06-01 00:03:00
2,6/1/2017 0:03,0,Bx10,E 206 ST/BAINBRIDGE AV,40.875008,-73.880142,RIVERDALE 263 ST,40.912376,-73.902534,NYCT_4223,...,-73.912647,HENRY HUDSON PKY E/W 235 ST,at stop,5,6/1/2017 0:03,00:00:53,NaN,6/1/2017,2017-06-01 00:00:53,2017-06-01 00:03:00
3,6/1/2017 0:03,0,Q5,TEARDROP/LAYOVER,40.701748,-73.802399,ROSEDALE LIRR STA via MERRICK,40.666012,-73.735939,NYCT_8422,...,-73.729348,HOOK CREEK BL/SUNRISE HY,< 1 stop away,267,6/1/2017 0:04,00:03:00,NaN,6/1/2017,2017-06-01 00:03:00,2017-06-01 00:04:00
4,6/1/2017 0:03,1,Bx1,RIVERDALE AV/W 231 ST,40.881187,-73.909340,MOTT HAVEN 136 ST via CONCOURSE,40.809654,-73.928360,NYCT_4710,...,-73.893032,GRAND CONCOURSE/E 196 ST,at stop,11,6/1/2017 0:03,23:59:38,NaN,6/1/2017,2017-06-01 23:59:38,2017-06-01 00:03:00


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1048575 entries, 0 to 1048574
Data columns (total 21 columns):
RecordedAtTime               1048575 non-null object
DirectionRef                 1048575 non-null int64
PublishedLineName            1048575 non-null object
OriginName                   1048575 non-null object
OriginLat                    1048575 non-null float64
OriginLong                   1048575 non-null float64
DestinationName              1048575 non-null object
DestinationLat               1048575 non-null float64
DestinationLong              1048575 non-null float64
VehicleRef                   1048575 non-null object
VehicleLocation.Latitude     1048575 non-null float64
VehicleLocation.Longitude    1048575 non-null float64
NextStopPointName            1048575 non-null object
ArrivalProximityText         1048575 non-null object
DistanceFromStop             1048575 non-null object
ExpectedArrivalTime          1048575 non-null object
ScheduledArrivalTime         10485

In [18]:
#filling in missing data
df['Scheduled_datetime'].ffill(axis=0, inplace=True)
df['Expected_datetime'].ffill(axis=0, inplace=True)


In [19]:
df.isnull().sum().sum()

1048516

In [20]:
#fixing the date value for those datetime entries where the hour is near midnight.
df['Scheduled_datetime'] = df['Scheduled_datetime'].apply(lambda x: x - timedelta(days=1) if str(x)[11:13] == '23' else x)

In [21]:
#create new feature to record the difference between each bus's scheduled arrival time and expected arrival time.
df['delay'] = df['Expected_datetime'] - df['Scheduled_datetime']

In [22]:
df.head()

,RecordedAtTime,DirectionRef,PublishedLineName,OriginName,OriginLat,OriginLong,DestinationName,DestinationLat,DestinationLong,VehicleRef,...,NextStopPointName,ArrivalProximityText,DistanceFromStop,ExpectedArrivalTime,ScheduledArrivalTime,Unnamed: 17,expected_date,Scheduled_datetime,Expected_datetime,delay
0,6/1/2017 0:03,0,B8,4 AV/95 ST,40.616104,-74.031143,BROWNSVILLE ROCKAWAY AV,40.656048,-73.907379,NYCT_430,...,FOSTER AV/E 18 ST,approaching,76,6/1/2017 0:03,00:06:14,NaN,6/1/2017,2017-06-01 00:06:14,2017-06-01 00:03:00,-1 days +23:56:46
1,6/1/2017 0:03,1,S61,ST GEORGE FERRY/S61 & S91,40.643169,-74.073494,S I MALL YUKON AV,40.575935,-74.167686,NYCT_8263,...,MERRYMOUNT ST/TRAVIS AV,approaching,62,6/1/2017 0:03,23:58:02,NaN,6/1/2017,2017-05-31 23:58:02,2017-06-01 00:03:00,00:04:58
2,6/1/2017 0:03,0,Bx10,E 206 ST/BAINBRIDGE AV,40.875008,-73.880142,RIVERDALE 263 ST,40.912376,-73.902534,NYCT_4223,...,HENRY HUDSON PKY E/W 235 ST,at stop,5,6/1/2017 0:03,00:00:53,NaN,6/1/2017,2017-06-01 00:00:53,2017-06-01 00:03:00,00:02:07
3,6/1/2017 0:03,0,Q5,TEARDROP/LAYOVER,40.701748,-73.802399,ROSEDALE LIRR STA via MERRICK,40.666012,-73.735939,NYCT_8422,...,HOOK CREEK BL/SUNRISE HY,< 1 stop away,267,6/1/2017 0:04,00:03:00,NaN,6/1/2017,2017-06-01 00:03:00,2017-06-01 00:04:00,00:01:00
4,6/1/2017 0:03,1,Bx1,RIVERDALE AV/W 231 ST,40.881187,-73.909340,MOTT HAVEN 136 ST via CONCOURSE,40.809654,-73.928360,NYCT_4710,...,GRAND CONCOURSE/E 196 ST,at stop,11,6/1/2017 0:03,23:59:38,NaN,6/1/2017,2017-05-31 23:59:38,2017-06-01 00:03:00,00:03:22


In [23]:
#extracting the number of seconds from 'delay' column to make new feature 'delay_second'
df['delay_second'] = df['delay'].dt.seconds

In [24]:
#creating new feature 'delay_minute' to convert the delay_second to minutes
df['delay_minute'] = df['delay_second'] / 60

In [25]:
#creating new feature delay_days by extracting the day value from 'delay' column
df['delay_days'] = df['delay'].dt.days

In [26]:
df.head()

,RecordedAtTime,DirectionRef,PublishedLineName,OriginName,OriginLat,OriginLong,DestinationName,DestinationLat,DestinationLong,VehicleRef,...,ExpectedArrivalTime,ScheduledArrivalTime,Unnamed: 17,expected_date,Scheduled_datetime,Expected_datetime,delay,delay_second,delay_minute,delay_days
0,6/1/2017 0:03,0,B8,4 AV/95 ST,40.616104,-74.031143,BROWNSVILLE ROCKAWAY AV,40.656048,-73.907379,NYCT_430,...,6/1/2017 0:03,00:06:14,NaN,6/1/2017,2017-06-01 00:06:14,2017-06-01 00:03:00,-1 days +23:56:46,86206,1436.766667,-1
1,6/1/2017 0:03,1,S61,ST GEORGE FERRY/S61 & S91,40.643169,-74.073494,S I MALL YUKON AV,40.575935,-74.167686,NYCT_8263,...,6/1/2017 0:03,23:58:02,NaN,6/1/2017,2017-05-31 23:58:02,2017-06-01 00:03:00,00:04:58,298,4.966667,0
2,6/1/2017 0:03,0,Bx10,E 206 ST/BAINBRIDGE AV,40.875008,-73.880142,RIVERDALE 263 ST,40.912376,-73.902534,NYCT_4223,...,6/1/2017 0:03,00:00:53,NaN,6/1/2017,2017-06-01 00:00:53,2017-06-01 00:03:00,00:02:07,127,2.116667,0
3,6/1/2017 0:03,0,Q5,TEARDROP/LAYOVER,40.701748,-73.802399,ROSEDALE LIRR STA via MERRICK,40.666012,-73.735939,NYCT_8422,...,6/1/2017 0:04,00:03:00,NaN,6/1/2017,2017-06-01 00:03:00,2017-06-01 00:04:00,00:01:00,60,1.000000,0
4,6/1/2017 0:03,1,Bx1,RIVERDALE AV/W 231 ST,40.881187,-73.909340,MOTT HAVEN 136 ST via CONCOURSE,40.809654,-73.928360,NYCT_4710,...,6/1/2017 0:03,23:59:38,NaN,6/1/2017,2017-05-31 23:59:38,2017-06-01 00:03:00,00:03:22,202,3.366667,0


In [27]:
df.loc[df['delay_days'] < 0, 'delay_minute'] = 0

In [28]:
df.head()

,RecordedAtTime,DirectionRef,PublishedLineName,OriginName,OriginLat,OriginLong,DestinationName,DestinationLat,DestinationLong,VehicleRef,...,ExpectedArrivalTime,ScheduledArrivalTime,Unnamed: 17,expected_date,Scheduled_datetime,Expected_datetime,delay,delay_second,delay_minute,delay_days
0,6/1/2017 0:03,0,B8,4 AV/95 ST,40.616104,-74.031143,BROWNSVILLE ROCKAWAY AV,40.656048,-73.907379,NYCT_430,...,6/1/2017 0:03,00:06:14,NaN,6/1/2017,2017-06-01 00:06:14,2017-06-01 00:03:00,-1 days +23:56:46,86206,0.000000,-1
1,6/1/2017 0:03,1,S61,ST GEORGE FERRY/S61 & S91,40.643169,-74.073494,S I MALL YUKON AV,40.575935,-74.167686,NYCT_8263,...,6/1/2017 0:03,23:58:02,NaN,6/1/2017,2017-05-31 23:58:02,2017-06-01 00:03:00,00:04:58,298,4.966667,0
2,6/1/2017 0:03,0,Bx10,E 206 ST/BAINBRIDGE AV,40.875008,-73.880142,RIVERDALE 263 ST,40.912376,-73.902534,NYCT_4223,...,6/1/2017 0:03,00:00:53,NaN,6/1/2017,2017-06-01 00:00:53,2017-06-01 00:03:00,00:02:07,127,2.116667,0
3,6/1/2017 0:03,0,Q5,TEARDROP/LAYOVER,40.701748,-73.802399,ROSEDALE LIRR STA via MERRICK,40.666012,-73.735939,NYCT_8422,...,6/1/2017 0:04,00:03:00,NaN,6/1/2017,2017-06-01 00:03:00,2017-06-01 00:04:00,00:01:00,60,1.000000,0
4,6/1/2017 0:03,1,Bx1,RIVERDALE AV/W 231 ST,40.881187,-73.909340,MOTT HAVEN 136 ST via CONCOURSE,40.809654,-73.928360,NYCT_4710,...,6/1/2017 0:03,23:59:38,NaN,6/1/2017,2017-05-31 23:59:38,2017-06-01 00:03:00,00:03:22,202,3.366667,0


In [29]:
#creating output variable
df['on_time'] = np.where((df['delay_minute'] > 5) | (df['delay_minute'] < -1), 0, 1)

In [30]:
df['on_time'].value_counts()

1    655212
0    393363
Name: on_time, dtype: int64

In [31]:
df['delay_days'].value_counts()

 0        764562
-1        271111
 1         12899
-10075         2
-17377         1
Name: delay_days, dtype: int64

In [ ]:
df_day_1 = df[df['delay_days'] == 1]

In [ ]:
df_day_1.head(50)

In [ ]:
#filtering dataframe for delay_days equal to -1, the buses that arrived before time.
df_day_neg1 = df[df['delay_days'] == -1]

In [ ]:
df_day_neg1.head(100)

In [ ]:
df_day_neg0 = df[df['delay_days'] == 0]

In [ ]:
df_day_neg0[4000:5000]

In [ ]:
#create a new dataframe by filtering the dataset for values of 'delay_neg'
df_neg = df[df['delay_neg']<0]

In [ ]:
df_neg.head()

In [ ]:
df_neg.shape

In [ ]:
df_neg.info()

In [ ]:
#df[df['delay_neg']<0] = 5

In [ ]:
#converting 'delay' column from timedelta to datetime
df['delay'] = pd.to_datetime(df['delay'], errors="coerce")

In [ ]:
df.head()

In [ ]:
#extracting the minutes from 'delay' column
df['delay_minute'] = df['delay'].dt.minute

In [ ]:
#checking datatype of 'delay_minute' column
df['delay_minute'].dtype

In [ ]:
#creating output variable
df['on_time'] = np.where((df['delay_minute'] > 5) | (df['delay_minute'] < -1), 0, 1)

In [ ]:
df.info()

In [ ]:
df["source"] = [(i, j) for i, j in zip(df["OriginLat"], df["OriginLong"])]

df["destination"] = [(i, j) for i, j in zip(df["DestinationLat"], df["DestinationLong"])]

df["dist"] = [geopy.distance.distance(i, j).km for i, j in zip(df["source"], df["destination"])]

In [ ]:
df['Vehicle_location2'] = [(i,j) for i, j in zip(df['VehicleLocation.Latitude'], df['VehicleLocation.Longitude'])]
df["VehicleToDestination"] = [geopy.distance.distance(i, j).km for i, j in zip(df["Vehicle_location2"], df["destination"])]

In [ ]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(timeout=3)
df['address_origin'] = [geolocator.reverse(i,j) for i, j in df['source']]

#df['location_address_origin'] = df['source'].apply(geolocator.reverse)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.PublishedLineName.value_counts()

In [ ]:
#using the 'on time' column and value counts to check how many buses were on time
df['on_time'].value_counts()

Out of a total of 1,048,575 recorded instances, 274,056 buses arrived on time and 774,519 buses were delayed.  This shows a significant class imbalance in our target variable that I will address before building a machine learning model from this data.

In [ ]:
# Resampling our data to correct for imbalance in the class variable 'on_time'

# Class count, 2 represents benign, and 4 represents malignant
count_ontime_0, count_ontime_1 = df.on_time.value_counts()
print(count_ontime_0)
print(count_ontime_1)
# Divide by class
df_ontime_0 = df[df['on_time'] == 0]
df_ontime_1 = df[df['on_time'] == 1]

In [ ]:
#extract the observations with class zero, equal to the number of obervations equal
df_ontime_0_under = df_ontime_0.sample(count_ontime_1)

#create a new dataframe with the same number of values for our two classes
new_df = pd.concat([df_ontime_0_under, df_ontime_1], axis=0)
new_df = new_df.reset_index()

print('Random under-sampling:')
print(new_df.on_time.value_counts())

new_df.on_time.value_counts().plot(kind='bar', title='Count (target)')

In [ ]:
#taking a quick glimpse of new_df
new_df.head()

In [ ]:
#exploring the 'OriginName' column
new_df.OriginName.describe()

It's important to know exactly how many different origin points we have in our data; by subsetting the "OriginName" column, we see that there are 457 unique origin points.  Next I will subset the data based on the two different values of the output variable, "on_time", to see if there is any relation between the Origin point of the bus and whether its on time or not.

Before performing any additional pre processing or feature creation, I will create a machine learing model with a decision tree to get an idea of the accuracy of the model at this point.

In [ ]:
#creating two subsets of the dataframe; new_df_0 represents the subset with buses not on time, and new_df_1
# represents the subset with buses on time.
new_df_0 = new_df[new_df['on_time'] == 0]
new_df_1 = new_df[new_df['on_time'] == 1]


In [ ]:
#exploring the PublishedLineName column
print(new_df_0.PublishedLineName.describe())
print(new_df_1.PublishedLineName.describe())


In [ ]:
#exploring the DestinationName column
print(new_df_0.DestinationName.describe())
print()
print(new_df_1.DestinationName.describe())

In [ ]:
#exploring the 'OriginName' column for those buses that did not make it to their destinations on time.
new_df_0.OriginName.describe()

In [ ]:
#exploring the 'OriginName' column for those buses that made it on time.
new_df_1.OriginName.describe()

Here I'm assuming the column "OriginName" to represent a bus stop, defined by the street intersection at which that bus stop is located at, from where each bus's route started, for each recorded observation in the data.

After subsetting the dataframe based on the binary values of the output variable, 'on_time', I noticed that the two subsets of the dataframe had similar number of unique "OriginNames; 448 out of 459 for the subset representing buses that were delayed and 446 out of 459 for the subset of data representing buses that were on time.  This suggests that both of the separate subsets had observations from almost all of the same bus stops, and that no one particular route was either always on time or always delayed.  Further analysis of the "OriginName" column below may help us see a pattern between the starting point (OriginName) of a bus's route and whether that bus was on time or not.

In [ ]:
#examing the top ten value_counts of dataframe new_df_0.
new_df_1.OriginName.value_counts().head(20)

In [ ]:
#examining the top ten value_counts of dataframe new_df_0
new_df_0.OriginName.value_counts().head(20)

After examining the two lists of "OriginName" value counts above, I noticed that the two lists from the separate subsets of the data share alot of the same bus stop names.  Bus stops (OriginName) such as E 126 ST/2 AV, ELTINGVILLE/TRANSIT CENTER, MERRICK BL/ARCHER AV, FLATBUSH AV/KINGS PLAZA, HARWAY AV/BAY 37 ST, and TEARDROP/LAYOVER were involved with alot of delays but also alot of buses that made it to their destination on time.  Considering that this data is regarding New York City, this may suggest that their is another underlying factor causing delays, such as high traffic on the roads due to seasonality or caused by special events at or near some of the specific bus stops that I mentioned here. 

Aside from this, it should be noted that of the 20 Origin names in the second list of value counts (which represents those OriginNames that were related to those buses being delayed) 11 are unique; that is they are not in the top 20 list of value counts for origin names that were related to those buses that were on time.  Names such as W FARMS RD/WESTCHESTER AV, BROADWAY/ISHAM ST, ASTOR PL/3 AV, AND 41 RD/MAIN ST, just to name a few.


In [ ]:
#making two scatter plots to show the origin locations of buses on time, and not on time, using origin latitude and origin
#longitude

#setting figure size
plt.figure(figsize=(15,10))

#making a plot figure that will have one row with two plots in each row
plt.subplot(1, 2, 1)
plt.scatter(x=new_df_0.OriginLat, y=new_df_0.OriginLong, color='blue')
plt.ylabel('Values')
plt.title('Origin Locations for buses Not On Time')

plt.subplot(1, 2, 2)
plt.scatter(x=new_df_1.OriginLat, y=new_df_1.OriginLong, color='blue')
plt.ylabel('Values')
plt.title('Origin Locations for buses On Time')
plt.show()

In [ ]:
#making two scatter plots to show the origin locations of buses on time, and not on time, using origin latitude and origin
#longitude

#setting figure size
plt.figure(figsize=(15,10))

#making a plot figure that will have one row with two plots in each row
plt.subplot(1, 2, 1)
plt.scatter(x=new_df_0.DestinationLat, y=new_df_0.DestinationLong, color='blue')
plt.ylabel('Values')
plt.title('Destination Locations for buses Not On Time')

plt.subplot(1, 2, 2)
plt.scatter(x=new_df_1.DestinationLat, y=new_df_1.DestinationLong, color='blue')
plt.ylabel('Values')
plt.title('Destination Locations for buses On Time')
plt.show()

The two sets of scatterplot show the origin locations and destination locations using latitude and longitude for all buses, on time and those not on time, from the dataset.  As you can see, they are near identical for both scenarios, which further states that the bus route in of itself was not the sole factor for causing a bus to be on time to its destination or delayed.

In [ ]:
new_df_0.DirectionRef.value_counts()

In [ ]:
new_df_1.DirectionRef.value_counts()

In [ ]:
new_df.head()

In [ ]:
new_df.info()

In [ ]:
df['dist'] = df['dist'].astype(float)

In [ ]:
df['dist'].dtype

In [ ]:
#checking dataframe to see which rows have non numeric data. False means its not int or float
#new_df.applymap(lambda x: isinstance(x, (int, float)))

In [ ]:
#converting 'DistanceFromStop' to numeric, and coercing errors which will make errors to nan, which i will correct below
#new_df['DistanceFromStop'] = pd.to_numeric(new_df['DistanceFromStop'], errors='coerce')

In [ ]:
#the total number of null items in "DistanceFromStop"
#new_df['DistanceFromStop'].isnull().sum()

In [ ]:
#rechecking the dtype of 'DistanceFromStop'
#new_df['DistanceFromStop'].dtype


In [ ]:
#checking the column shape
#new_df['DistanceFromStop'].shape

In [ ]:
#this just gave me a long pandas series
#print(np.isnan(new_df['DistanceFromStop']))

In [ ]:
#one way to check which rows have the nan or null values in "DistanceFromStop" column, by subsetting database for null values
#print(new_df[pd.to_numeric(new_df['DistanceFromStop'], errors='coerce').isnull()])

In [ ]:
#dropping the Nan values.  I could have also replaced the NaN values with with fillna(0)
#new_df = new_df.drop(new_df.index[[53965,56430,61000,65636]])

In [ ]:
#rechecking the shape of the 'DistanceFromStop' column
#new_df['DistanceFromStop'].shape

In [ ]:
new_df.columns

In [ ]:
#dropping some columns and splitting the data and creating the initial train and test splits.
X = new_df.drop(['index', 'RecordedAtTime', 'ArrivalProximityText', 'Unnamed: 17', 'PublishedLineName', 'VehicleRef', 
                 'NextStopPointName', 'ExpectedArrivalTime', 'ScheduledArrivalTime',
                 'expected_arrival', 'scheduled_arrival', 'delay', 'delay_minute', 'on_time', 'OriginName',
                 'DestinationName','source', 'destination', 'Vehicle_location2', 'DistanceFromStop'], 1)

#getting dummy values for categorical data, and creating dataframe of the independent (predictor) values
#XD_df = pd.get_dummies(X, columns=['OriginName', 'DestinationName'])


Y = new_df['on_time']



In [ ]:
X.head(20)

In [ ]:
#checking type
print(type(X))

In [ ]:
#checking which column has NaN value
X.isnull().sum().sum()

In [ ]:
#checking for location of NaN values in "DistanceFromStop" column
#print(XD_df[pd.to_numeric(XD_df['DistanceFromStop'], errors='coerce').isnull()])

In [ ]:
#dropping the Nan values.  I could have also replaced the NaN values with with fillna(0)
#XD_df = XD_df.drop(XD_df.index[4606])
#Y = Y.drop(Y.index[4606])

In [ ]:
X.shape


In [ ]:
Y.shape

In [ ]:
train_X, test_X, train_Y, test_Y = train_test_split(X, Y, test_size=.3, random_state=20)
print(test_X.shape)
print(test_Y.shape)

#checking for any NaN values in our data.
#print(np.isnan(train_Y).any())
#print(np.isnan(train_X).any().sum())
#print(np.isnan(test_Y).any())
#print(np.isnan(test_X).any().sum())

In [ ]:
# perform grid search to find the optimal parameters for our decision tree model
dtree = tree.DecisionTreeClassifier(random_state=2)
tree_param_grid = {'max_depth':[4,6,10], 'min_samples_split':[6,8,10,15], 'max_features':[2,4,6,8], 
                   'criterion':['gini', 'entropy']}
dtree_grid = GridSearchCV(dtree, tree_param_grid, cv=5, verbose=3)
dtree_grid.fit(train_X, train_Y)
print('Best parameters for decision tree:', dtree_grid.best_params_)

In [ ]:
print('Best parameters for decision tree:', dtree_grid.best_params_)
print('Best score for decision tree: ', dtree_grid.best_score_)

In [ ]:
# Create the first model by Initializing and training our tree before any additional preprocessing
decision_tree = tree.DecisionTreeClassifier(
    criterion='gini',
    max_features=4,
    max_depth=10,
    min_samples_split=8,
    random_state = 201
)

#set start time
start_time_tree = datetime.datetime.now()

#train our tree
decision_tree.fit(train_X, train_Y)

# Classify, storing the result in a new variable.
y_pred_dtree = decision_tree.predict(test_X)

#get accuracy score from our model
print('Accuracy score for decision tree Model: ' + str(metrics.accuracy_score(test_Y, y_pred_dtree)))

#calculate end time
end_time_tree = datetime.datetime.now()

#print the execution run time for our model
print('\nDuration: {}'.format(end_time_tree - start_time_tree))
print()

#get accuracy score of decision tree
tree_score = cross_val_score(decision_tree, X, Y, cv=10, scoring='accuracy')
print(tree_score)
print('Average cross validated score from our decision tree model is: ' + str(np.mean(tree_score)))

In [ ]:
kbest = SelectKBest(f_classif)
pipeline = Pipeline([('kbest', kbest), ('lr', LogisticRegression())])
gridcv_k_log = GridSearchCV(pipeline, {'kbest__k': [1,2,3,4], 'lr__C': np.logspace(-10, 10, 5)}, verbose=3)
gridcv_k_log.fit(train_X, train_Y)

In [ ]:
print('Best parameters for k_log:', gridcv_k_log.best_params_)
print('Best score for k_log: ', gridcv_k_log.best_score_)

# Logistic Regression

In [ ]:
#initialize and train our model
LogReg = LogisticRegression(C=1e9, random_state=12)

#set start time
start_time4 = datetime.datetime.now()

#fit and train the model
LogReg.fit(train_X, train_Y)

# Classify, storing the result in a new variable.
y_pred_LogReg = LogReg.predict(test_X)

#get accuracy score from our model
print('Accuracy score for Logistic regression Model: ' + str(metrics.accuracy_score(test_Y, y_pred_LogReg)))

#calculate end time
end_time4 = datetime.datetime.now()

#print the execution run time for our model
print('\nDuration: {}'.format(end_time4 - start_time4))
print()

#get average accuracy score using cross validation
LogReg_cvs = cross_val_score(LogReg, X, Y, cv=10, scoring='accuracy')
print(LogReg_cvs)

print('Average cross validated score from our log regression model is: ' + str(np.mean(LogReg_cvs)))

# PCA

In [ ]:
# Normalize the data so that all variables have a mean of 0 and standard deviation
# o

X_scaled = StandardScaler().fit_transform(X)

In [ ]:
#Apply PCA

pca = PCA(.95)
X_pca = pca.fit_transform(X_scaled)

comp_no = pca.n_components_
comp_var_ratio = pca.explained_variance_ratio_

print(comp_no)
print(comp_var_ratio)
#print(p_components)

In [ ]:
#pca data to dataframe
pca_df = pd.DataFrame(data=X_pca)
print(pca_df.head())
print(pca_df.shape)

In [ ]:
#After applying PCA, I split the data into a second set of train and test groups

train_X_pca, test_X_pca, train_Y_pca, test_Y_pca = train_test_split(X_pca, Y, test_size=.3, random_state=20)

# Random Forest

In [ ]:
# perform grid search to find the optimal parameters for our random forest model, after applying PCA
rforest = ensemble.RandomForestClassifier(random_state=20)
forest_param_grid = {'n_estimators':[50,100], 'max_depth':[4,6,8,10], 'min_samples_split':[6,8,10,12], 
                     'max_features':[2,4,6], 'criterion':['entropy', 'gini']}
rforest_grid = GridSearchCV(rforest, forest_param_grid, cv=10, verbose=3)
rforest_grid.fit(train_X, train_Y)
print('Best parameters for decision tree:', rforest_grid.best_params_)

In [ ]:
print('Best score for Random Forest:', rforest_grid.best_score_)

In [ ]:
#initialize and train our random forest model with PCA
rand_forest_pca = ensemble.RandomForestClassifier(n_estimators=100, criterion='gini', max_depth=10, max_features=6,
                                              min_samples_split=10, random_state=300)

#set start time
start_time_rforest = datetime.datetime.now()

rand_forest_pca.fit(train_X_pca, train_Y_pca)

# Classify, storing the result in a new variable.
y_pred_rforest_pca = rand_forest_pca.predict(test_X_pca)

#get accuracy score from our model
print('Accuracy score for Random Forest Model: ' + str(metrics.accuracy_score(test_Y_pca, y_pred_rforest)))

#calculate end time
end_time_rforest = datetime.datetime.now()

#print the execution run time for our model
print('\nDuration: {}'.format(end_time_rforest - start_time_rforest))
print()

#get average accuracy score of our random forest through cross validation
rforest_pca_cvs = cross_val_score(rand_forest_pca, X_pca, Y, cv=10, scoring='accuracy')
print(rforest_pca_cvs)
print('Average cross validated score from our Random Forest Model is: ' + str(np.mean(rforest_pca_cvs)))

In [ ]:
# create confusion matrix for random forest and PCA
confusion_matrix(test_Y_pca, y_pred_rforest_pca)

In [ ]:
#Get AUC and create ROC
# predict probabilities
probs_rforest_pca = LogReg.predict_proba(test_X_pca)
# keep probabilities for the positive outcome only
probs_rforest_pca = probs_rforest_pca[:, 1]
# calculate AUC
rforest_auc = roc_auc_score(test_Y_pca, probs_rforest_pca)
print('AUC Score: %.3f' % rforest_auc)
# calculate roc curve
fpr_rforest_pca, tpr_rforest_pca, thresholds_rforest_pca = roc_curve(test_Y_pca, probs_rforest_pca)
# plot no skill
plt.plot([0, 1], [0, 1], linestyle='--')
# plot the roc curve for the model
plt.plot(fpr_rforest_pca, tpr_rforest_pca, marker='.')
# show the plot
plt.show()

In [ ]:
# perform classification report of the logistic regression model
class_names = ['Class_0', 'Class_1']
rforest_report = classification_report(test_Y_pca, y_pred_rforest_pca, target_names=class_names)
print(rforest_report)

# Gradient Boosting

In [ ]:
#Run gridsearchcv to find the best parameters for gradient boosting using PCA
gboost_param_grid = {'n_estimators':[200,400,600], 'max_depth':[2,4,6], 'min_samples_leaf': [3, 5, 9], 
                     'learning_rate': [.05, .02, .01], 'subsample': [.7, .8, .9]}
gb_model = ensemble.GradientBoostingClassifier(random_state=67)

gb_grid = GridSearchCV(gb_model, gboost_param_grid, cv=5)
gb_grid.fit(train_X_pca, train_Y_pca)
print('Best parameters for Gradient Boost Model are:', gb_grid.best_params_)

In [ ]:
print('Best parameters for Gradient Boost Model are:', gb_grid.best_params_)
print('Best score for Gradient Boost Model are:', gb_grid.best_score_)

In [ ]:
# Initialize and fit the model using PCA
Grad_boost_pca = ensemble.GradientBoostingClassifier(learning_rate=0.02, max_depth=2, min_samples_leaf=5, 
                                                 n_estimators=200, subsample=0.7, random_state=67)

#set start time
start_time_gb = datetime.datetime.now()

#fit and train the model
Grad_boost_pca.fit(train_X_pca, train_Y_pca)

# Classify, storing the result in a new variable.
y_pred_Gboost_pca = Grad_boost.predict(test_X_pca)

#get accuracy score from our model
print('Accuracy score for Gradient Boost Model: ' + str(metrics.accuracy_score(test_Y_pca, y_pred_Gboost_pca)))

#calculate end time
end_time_gb = datetime.datetime.now()

#print the execution run time for our model
print('\nDuration: {}'.format(end_time_gb - start_time_gb))
print()

#get average accuracy score using cross validation
Grad_boost_cvs_pca = cross_val_score(Grad_boost_pca, X_pca, Y, cv=5, scoring='accuracy')
print(Grad_boost_cvs_pca)

print('Average cross validated score from our Gradient Boosting model is: ' + str(np.mean(Grad_boost_cvs_pca)))

In [ ]:
#Get AUC and create ROC
# predict probabilities
probs_Gboost_pca = Grad_boost_pca.predict_proba(test_X_pca)
# keep probabilities for the positive outcome only
probs_Gboost_pca = probs_Gboost_pca[:, 1]
# calculate AUC
Gboost_auc = roc_auc_score(test_Y_pca, probs_Gboost_pca)
print('AUC Score: %.3f' % Gboost_auc)
# calculate roc curve
fpr_Gboost_pca, tpr_Gboost_pca, thresholds_Gboost_pca = roc_curve(test_Y_pca, probs_Gboost_pca)
# plot no skill
plt.plot([0, 1], [0, 1], linestyle='--')
# plot the roc curve for the model
plt.plot(fpr_Gboost_pca, tpr_Gboost_pca, marker='.')
# show the plot
plt.show()

print()
# perform classification report of the Gradient Boosting model
class_names = ['Class_0', 'Class_1']
Gboost_report = classification_report(test_Y_pca, y_pred_Gboost_pca, target_names=class_names)
print(Gboost_report)
print()
#confusion matrix
print('Confusion Matrix:')
confusion_matrix(test_Y_pca, y_pred_Gboost_pca)


# Logistic Regression

In [ ]:
#initialize and train our model after PCA
LogReg_pca = LogisticRegression(C=1e9, random_state=12)

#set start time
start_time4 = datetime.datetime.now()

#fit and train the model
LogReg_pca.fit(train_X_pca, train_Y_pca)

# Classify, storing the result in a new variable.
y_pred_LogReg_pca = LogReg_pca.predict(test_X_pca)

#get accuracy score from our model
print('Accuracy score for Logistic regression Model after pca: ' + str(metrics.accuracy_score(test_Y_pca, y_pred_LogReg_pca)))

#calculate end time
end_time4 = datetime.datetime.now()

#print the execution run time for our model
print('\nDuration: {}'.format(end_time4 - start_time4))
print()

#get average accuracy score using cross validation
LogReg_cvs_pca = cross_val_score(LogReg_pca, X_pca, Y, cv=10, scoring='accuracy')
print(LogReg_cvs_pca)

print('Average cross validated score from our log regression model is: ' + str(np.mean(LogReg_cvs_pca)))

In [ ]:
#Get AUC and create ROC
# predict probabilities
probs_LogReg_pca = LogReg_pca.predict_proba(test_X)
# keep probabilities for the positive outcome only
probs_LogReg_pca = probs_LogReg_pca[:, 1]
# calculate AUC
LogReg_pca_auc = roc_auc_score(test_Y_pca, probs_LogReg_pca)
print('AUC Score: %.3f' % LogReg_pca_auc)
# calculate roc curve
fpr_LogReg_pca, tpr_LogReg_pca, thresholds_LogReg_pca = roc_curve(test_Y_pca, probs_LogReg_pca)
# plot no skill
plt.plot([0, 1], [0, 1], linestyle='--')
# plot the roc curve for the model
plt.plot(fpr_LogReg_pca, tpr_LogReg_pca, marker='.')
# show the plot
plt.show()
print()

# perform classification report of the logistic regression model
class_names = ['Class_0', 'Class_1']
LogReg_report_pca = classification_report(test_Y_pca, y_pred_LogReg_pca, target_names=class_names)
print(LogReg_report_pca)
print()

# create confusion matrix
confusion_matrix(test_Y_pca, y_pred_LogReg_pca)

# KNN

In [ ]:
# perform grid search to find the optimal parameters for our KNN model using full data.
KNN_model = KNeighborsClassifier()
KNN_params = {'n_neighbors':[5,10,15], 'weights':['uniform', 'distance']}
KNN_grid = GridSearchCV(KNN_model, KNN_params, cv=5)
KNN_grid.fit(train_X_pca, train_Y_pca)
print('Best parameters for data:', KNN_grid.best_params_)

In [ ]:
#initialize and train our model using the training set
KNN_pca = KNeighborsClassifier(n_neighbors=5, weights='distance')

#set start time
starttime_knn = datetime.datetime.now()

#fit and train the model
KNN_pca.fit(train_X_pca,train_Y_pca)

#Make predictions using test_X, and store results in new variable
y_pred_KNN_pca = KNN_pca.predict(test_X_pca)

print('Accuracy score for KNN Model: ' + str(metrics.accuracy_score(test_Y_pca, y_pred_KNN_pca)))

#calculate end time
endtime_knn = datetime.datetime.now()

#print the execution run time for our model
print('\nDuration: {}'.format(endtime_knn - starttime_knn))
print()


KNN_cvs_pca = cross_val_score(KNN_pca, X_pca, Y, cv=5, scoring='accuracy')
print(KNN_cvs_pca)
print('Average cross validated score from our KNN model is: ' + str(np.mean(KNN_cvs_pca)))


In [ ]:
#Get AUC and create ROC
# predict probabilities
probs_KNN_pca = KNN_pca.predict_proba(test_X_pca)
# keep probabilities for the positive outcome only
probs_KNN_pca = probs_KNN_pca[:, 1]
# calculate AUC
KNN_pca_auc = roc_auc_score(test_Y_pca, probs_KNN_pca)
print('AUC Score: %.3f' % KNN_pca_auc)
# calculate roc curve
fpr_KNN_pca, tpr_KNN_pca, thresholds_KNN_pca = roc_curve(test_Y_pca, probs_KNN_pca)
# plot no skill
plt.plot([0, 1], [0, 1], linestyle='--')
# plot the roc curve for the model
plt.plot(fpr_KNN_pca, tpr_KNN_pca, marker='.')
# show the plot
plt.show()

# perform classification report of the KNN model
class_names = ['Class_0', 'Class_1']
KNN_report_pca = classification_report(test_Y_pca, y_pred_KNN_pca, target_names=class_names)
print(KNN_report_pca)

# confusion Matrix
confusion_matrix(test_Y_pca, y_pred_KNN_pca)


# SelectKBest

In [ ]:
kbest = SelectKBest(f_classif)
pipeline = Pipeline([('kbest', kbest), ('lr', LogisticRegression())])
grid_search = GridSearchCV(pipeline, {'kbest__k': [1,2,3,4], 'lr__C': np.logspace(-10, 10, 5)})
grid_search.fit(train_X_pca, train_Y_pca)

In [ ]:
#performing GridsearchCV to get the best parameters for SelectKBest.
#Kbest = SelectKBest().fit_transform(X, Y)
#K_params = {'k':[1,2,3,4,5,6], 'score_func':[chi2, f_classif]}
#Kgrid = GridSearchCV(Kbest, K_params, cv=5, scoring='accuracy')
#Kgrid.fit(X, Y)
#print('Best parameters for SelectKBest:', Kgrid.best_params_)

In [ ]:
#applying selectKBest.
X_skb = SelectKBest(f_classif, k=6).fit_transform(X, Y)
print(X_skb.shape)

In [ ]:
#After applying SelectKBest, I split the data into a third set of train and test groups
train_X_skb, test_X_skb, train_Y_skb, test_Y_skb = train_test_split(X_skb, Y, test_size=.3, random_state=20)

In [ ]:
#an unconventional way to check for the null, nan, or NaT value in scheduled_arrival
#df['checkNaN'] = df['scheduled_arrival'].isnull()
#print(np.where(df['checkNaN'] == True)) 

#plt.figure(figsize=(10,10))
#sns.jointplot(x=new_df_0.OriginLat, y=new_df_0.OriginLong, size=10)
#plt.ylabel('Longitude', fontsize=12)
#plt.xlabel('Latitude', fontsize=12)
#plt.show()
#sns.despine

#plt.figure(figsize=(10,10))
#sns.jointplot(x=new_df_1.OriginLat, y=new_df_1.OriginLong, size=10)
#plt.ylabel('Longitude', fontsize=12)
#plt.xlabel('Latitude', fontsize=12)
#plt.show()
#sns.despine

In [ ]:
#df.replace(to_replace=df['scheduled_arrival'].dt.day, value=df['expected_arrival'].dt.day, inplace=True)

#df['ScheduledArrivalTime'].apply(lambda x: x[0:2].replace('24', '00'))

#examining the head of the 'expected_arrival' column.
#print(df['expected_arrival'].head())

#df['expected_arrival'].dt.day.value_counts()

#examining the head of the 'scheduled_arrival' column.
#print(df['scheduled_arrival'].head())

#df['scheduled_arrival'].dt.day.value_counts()

#dayVal = df['expected_arrival'].dt.day
#print(type(dayVal))

#df['scheduled_arrival'] = df['scheduled_arrival'].apply(lambda t: t.replace(day=Var))

In [ ]:
#creating new features by converting strings to datetimes
#df['expected_arrival'] = pd.to_datetime(df['ExpectedArrivalTime'], errors="coerce")
#df['scheduled_arrival'] = pd.to_datetime(df['ScheduledArrivalTime'], errors="coerce")

In [ ]:
#correcting the year value in the two new feature columns
#df['expected_arrival'] = df['expected_arrival'].apply(lambda t: t.replace(year=2017) if not pd.isnull(t) else pd.NaT)
#df['scheduled_arrival'] = df['scheduled_arrival'].apply(lambda t: t.replace(year=2017) if not pd.isnull(t) else pd.NaT)

#correcting the day value in the new feature columns
#df['expected_arrival'] = df['expected_arrival'].apply(lambda t: t.replace(day=1) if not pd.isnull(t) else pd.NaT)
#df['scheduled_arrival'] = df['scheduled_arrival'].apply(lambda t: t.replace(day=1) if not pd.isnull(t) else pd.NaT)

#correcting the month value in the new feature columns
#df['expected_arrival'] = df['expected_arrival'].apply(lambda t: t.replace(month=6) if not pd.isnull(t) else pd.NaT)
#df['scheduled_arrival'] = df['scheduled_arrival'].apply(lambda t: t.replace(month=6) if not pd.isnull(t) else pd.NaT)
#print(df['expected_arrival'])


#df['delay_neg'] = pd.to_numeric(df['delay'], errors='coerce')